# Segmenting and Clustering neighborhood in Toronto

[Toronto neighborhood data scrapped from Wiki](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [1]:
import pandas as pd
import numpy as np

In [86]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
result = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = result[0]

In [3]:
#Filtering all the Un Assigned postal codes
df=df[df['Borough']!='Not assigned'].reset_index(drop=True)
print(df.shape)
df.head()

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df.shape

(103, 3)

# Latitude and Longitude

In [19]:
#Loading data from a CSV file
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head(2)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


In [18]:
#Merging dataframes on Postal Code column
toronto_city_data = pd.merge(df, df_geo, on="Postal Code") #SIZE -> (103,5)
toronto_city_data.head(2)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572


In [23]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)
toronto_map

In [22]:
# add markers to map
for lat, lng, label in zip(toronto_city_data['Latitude'], toronto_city_data['Longitude'], toronto_city_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

# Explore Neighborhood via Foursquare API

In [25]:
import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files

# Foursquare Details
<!--
CLIENT_ID = 'WUZY40MF4YNPZISXVAKKHHR0N3XJP3IST2ESZ2XZQRKMS3YV5IF' # your Foursquare ID
CLIENT_SECRET = '5WYNNCKSNRSUVM012QGJM3KAYLCUH03DWLRPPPRB5AKKLWV5HJF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
-->

In [91]:
def get_category_name(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    elif isinstance(categories_list, str) or (categories_list is None):
        return categories_list
    else:
        return categories_list[0]['name']

In [114]:
def getNearbyVenues(neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    
    for neighborhood, lat, lng in zip(neighborhoods, latitudes, longitudes):
        #print(neighborhood)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    return venues_list

In [93]:
toronto_venues_list = getNearbyVenues(neighborhoods=toronto_city_data['Neighborhood'],
                                   latitudes=toronto_city_data['Latitude'],
                                   longitudes=toronto_city_data['Longitude']
                                  )

In [149]:
toronto_venues = pd.DataFrame([item for venue_list in toronto_venues_list for item in venue_list]) #SIZE -> (2163, 7)
toronto_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

In [150]:
print(toronto_venues.shape)
toronto_venues.head()

(2163, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [180]:
print(len(toronto_venues.groupby('Neighborhood'))) # Size -> 95
toronto_venues.groupby('Neighborhood').count()

95


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",28,28,28,28,28,28
Berczy Park,57,57,57,57,57,57
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


In [152]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique()))) # 274 Unique Categories

There are 274 uniques categories.


# Onehot encoding for categorical variable

In [153]:
toronto_venues_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="") # size -> (2163, 274)
toronto_venues_onehot.shape

(2163, 274)

In [157]:
toronto_venues_onehot.rename(columns={'Neighborhood':'neighborhood'}, inplace=True) # There is one entry with same name as a real column name
toronto_venues_onehot['Neighborhood'] = toronto_venues['Neighborhood'] # size -> (2163, 275)

# move neighborhood column to the first column
fixed_columns = [toronto_venues_onehot.columns[-1]] + list(toronto_venues_onehot.columns[:-1])
toronto_venues_onehot = toronto_venues_onehot[fixed_columns]

toronto_venues_onehot.head()


,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,neighborhood,neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,neighborhood,neighborhood
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Groupby Neighbourhood and Calculate mean of occurance of each category

In [179]:
#Group neighbourhood and calculate mean of occurance of each category, each occurance is represented as 1 in onehot encoding
toronto_venues_grouped = toronto_venues_onehot.groupby('Neighborhood').mean().reset_index()
toronto_venues_grouped.head(2)
len(toronto_venues_grouped) # Size -> 95

95

### Calculate top X common venues in each Neighborhood

In [218]:
#Calculate top 3 common venues
top_venues = 3
for neighbourhood in toronto_venues_grouped['Neighborhood'].head(1):
    print("-----"+neighbourhood+"-----")
    selected_neighborhood = toronto_venues_grouped[toronto_venues_grouped['Neighborhood'] == neighbourhood].T.reset_index()
    selected_neighborhood.columns = ['venue','frequency']
    selected_neighborhood = selected_neighborhood.iloc[1:]
    selected_neighborhood['frequency'] = selected_neighborhood['frequency'].astype(float)
    selected_neighborhood = selected_neighborhood.round({'frequency': 2})
    print(selected_neighborhood.sort_values('frequency', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

-----Agincourt-----
                       venue  frequency
0  Latin American Restaurant       0.25
1                     Lounge       0.25
2               Skating Rink       0.25




In [219]:
def get_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

In [278]:
top_venues = 10

numerical_indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for venue_place in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(venue_place+1, numerical_indicators[venue_place])) # Outof Index error expected
    except:
        columns.append('{}th Most Common Venue'.format(venue_place+1))

# create a new dataframe
neighborhoods_venues_sorted_by_occurance = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_by_occurance['Neighborhood'] = toronto_venues_grouped['Neighborhood']

for index in np.arange(toronto_venues_grouped.shape[0]):
    neighborhoods_venues_sorted_by_occurance.iloc[index, 1:] = get_most_common_venues(toronto_venues_grouped.iloc[index, :], top_venues)

print(neighborhoods_venues_sorted_by_occurance.shape) #SIZE -> (95,9)
neighborhoods_venues_sorted_by_occurance.head()

(95, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Convention Center,Doner Restaurant,Fast Food Restaurant,Farmers Market,Farm,Construction & Landscaping
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pub,Gym,Sandwich Place,Coffee Shop,Diner,Discount Store,Distribution Center,Dog Run
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Mobile Phone Shop,Supermarket,Deli / Bodega,Sushi Restaurant,Diner,Fried Chicken Joint,Frozen Yogurt Shop
3,Bayview Village,Bank,Café,Chinese Restaurant,Japanese Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,neighborhood
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Juice Bar,Restaurant,Italian Restaurant,Coffee Shop,Greek Restaurant,Thai Restaurant,Liquor Store,Boutique,Fast Food Restaurant


# Cluster   *k*-means

In [279]:
from sklearn.cluster import KMeans
import numpy as np

In [280]:
# set number of clusters
k_clusters = 6

toronto_grouped_clustering = toronto_venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
k_means = KMeans(init='k-means++', n_clusters=k_clusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
k_means.labels_ #Size -> 95

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3,
       0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       3, 0, 0, 0, 0, 0, 3])

In [281]:
#neighborhoods_venues_sorted_by_occurance.insert(0, 'Cluster Labels', k_means.labels_) #SIZE -> (95,9)
#neighborhoods_venues_sorted_by_occurance.head()


In [282]:

# add clustering labels to sorted occurance of venues
neighborhoods_venues_sorted_by_occurance.insert(0, 'Cluster Labels', k_means.labels_) #SIZE -> (95,9)

toronto_city_merged = toronto_city_data #Initial Main data , SIZE -> (103,5)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#  #SIZE -> (103,5) join #SIZE -> (95,9)
toronto_city_merged = toronto_city_merged.join(neighborhoods_venues_sorted_by_occurance.set_index('Neighborhood'), on='Neighborhood')

#toronto_city_merged[['Cluster Labels']] = toronto_city_merged[['Cluster Labels']].applymap(np.int64)
toronto_city_merged[['Cluster Labels']] = toronto_city_merged[['Cluster Labels']].fillna(0.0).astype(int) #fix for Buggy join operation converting int to flot
print(toronto_city_merged.shape) #SIZE -> (103,14)
toronto_city_merged.head() # check the last columns!


(103, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Bus Stop,Park,Food & Drink Shop,neighborhood,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Portuguese Restaurant,Coffee Shop,Pizza Place,Hockey Arena,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Theater,Pub,Beer Store,Dessert Shop,Event Space
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Miscellaneous Shop,Boutique,Vietnamese Restaurant,Women's Store,Cosmetics Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,College Cafeteria,Diner,Gym,Café,Sushi Restaurant,Fried Chicken Joint,Beer Bar,Italian Restaurant,Burrito Place


# Visualize Clusters

In [283]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [288]:
# create map
toronto_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(toronto_city_merged['Latitude'], toronto_city_merged['Longitude'], toronto_city_merged['Neighborhood'], toronto_city_merged['Cluster Labels']):
    label = folium.Popup(str(neighbourhood) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)
       
toronto_map_clusters

# Explore Cluster

In [294]:
toronto_city_merged.loc[toronto_city_merged['Cluster Labels'] == 3 , toronto_city_merged.columns[[1] + list(range(5, toronto_city_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,3,Park,Women's Store,Pool,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
35,East York,3,Park,Convenience Store,Coffee Shop,neighborhood,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
61,Central Toronto,3,Park,Bus Line,Swim School,neighborhood,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
64,York,3,Park,Convenience Store,neighborhood,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
66,North York,3,Electronics Store,Park,Convenience Store,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
85,Scarborough,3,Park,Playground,Bakery,neighborhood,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
91,Downtown Toronto,3,Park,Trail,Playground,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
101,Etobicoke,3,Park,Construction & Landscaping,Baseball Field,neighborhood,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


# Help!
<!-- 
# Group by postal code, transform neighborhood to single value where they share same postal code
#df['Neighborhood'] = df.groupby(['Postal Code'])['Neighborhood'].transform(lambda x: ','.join(x))
#df.drop_duplicates(inplace=True)

#df = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join).reset_index()
#df['Postal Code'].drop_duplicates(inplace=True)

#Unique values for categorical variables
#df['Postal Code'].value_counts().rename_axis('Postal Code').reset_index(name='Counts')
-->